Approximating CDFs for Nash distribution using "fast" fictitious play

In [9]:
using Random
using DataFrames
using CSV
using Statistics
using Printf
using Distributions
using LinearAlgebra

struct Fighter
    name::String
    h::Int16
    f::Int16
    l::Int16
    t::Int16
end

const MXS = 100
const MXV = 20
const MNV = 1

function random_fighter(h0::Int = MNV, f0::Int = MNV, l0::Int = MNV, t0::Int = MNV)::Fighter
    flag = true
    while flag
        h = h0 + rand(0:(MXV - h0), 1)[1]
        f = f0 + rand(0:(MXV - f0), 1)[1]
        l = l0 + rand(0:(MXV - l0), 1)[1]
        t = t0 + rand(0:(MXV - t0), 1)[1]
        mm = minimum([h, f])
        budget = h*f+h*l+f*t
        if budget <= MXS && budget + mm > MXS
            flag = false
            return Fighter("No Name", h, f, l, t)
        end
    end
end

function eval_battle(a::Fighter, b::Fighter)::Int
    a_finds = a.f >= b.h
    b_finds = b.f >= a.h
    if a_finds && b_finds
        if a.t > b.t
            return 1
        end
        if a.t < b.t
            return -1
        end
        if a.t == b.t
            if a.l > b.l
                return 1
            end
            if a.l < b.l
                return -1
            end
            if a.l == b.l
                if a.h > b.h
                    return 1
                end
                if a.h < b.h
                    return -1
                end
                if a.h == b.h
                    return 0
                end
            end
        end
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        if a.l > b.l
            return 1
        end
        if a.l < b.l
            return -1
        end
        if a.l == b.l
            if a.t > b.t
                return 1
            end
            if a.t < b.t
                return -1
            end
            if a.t == b.t
                if a.f > b.f
                    return 1
                end
                if a.f < b.f
                    return -1
                end
                if a.f == b.f
                    return 0
                end
            end
        end
    end
end

function random_tournament_winner(c::Int, f::Function = random_fighter)::Fighter
    if c==0
        return f()
    end
    a = random_tournament_winner(c-1, f)
    b = random_tournament_winner(c-1, f)
    res = eval_battle(a, b)
    if res == 1
        return a
    else
        return b
    end
end


tab = CSV.read("census_yob2022_names.txt", DataFrame, header = false)
names = tab.Column1
adjectives = CSV.read("adjectives.csv", DataFrame)
nouns = CSV.read("nouns.csv", DataFrame)

function random_name_and_stat()::Fighter
    vp = [0, 0, 0, 0]
    nametype = rand(1:5)
    name = ""
    if nametype == 1 || nametype == 2
        nm = rand(names)
        adj_i = rand(1:nrow(adjectives))
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H]
        vp[2] = vp[2] + adjectives[adj_i, :F]
        vp[3] = vp[3] + adjectives[adj_i, :L]
        vp[4] = vp[4] + adjectives[adj_i, :T]
        if nametype == 1
            name = string(nm, " the ", adj)
        end
        if nametype == 2
            name = string(adj, " ", nm)
        end
    end
    if nametype == 3 || nametype == 4
        nm = rand(names)
        noun_i = rand(1:nrow(nouns))
        noun = nouns[noun_i, :noun]
        vp[1] = vp[1] + nouns[noun_i, :H]
        vp[2] = vp[2] + nouns[noun_i, :F]
        vp[3] = vp[3] + nouns[noun_i, :L]
        vp[4] = vp[4] + nouns[noun_i, :T]    
        if nametype == 3
            name = string(nm, " the ", noun)
        end
        if nametype == 4
            name = string(noun, " ", nm)
        end
    end
    if nametype == 5
        adj_i = rand(1:nrow(adjectives))
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H]
        vp[2] = vp[2] + adjectives[adj_i, :F]
        vp[3] = vp[3] + adjectives[adj_i, :L]
        vp[4] = vp[4] + adjectives[adj_i, :T]
        noun_i = rand(1:nrow(nouns))
        noun = nouns[noun_i, :noun]
        vp[1] = vp[1] + nouns[noun_i, :H]
        vp[2] = vp[2] + nouns[noun_i, :F]
        vp[3] = vp[3] + nouns[noun_i, :L]
        vp[4] = vp[4] + nouns[noun_i, :T]    
        name = string(adj, " ", noun)
    end
    Fighter(name, vp[1], vp[2], vp[3], vp[4])
end

function rand_rename(a::Fighter, n_tries::Int = 10)::Fighter
    best_score = 0.0
    best_b = random_name_and_stat()
    for ii in 1:n_tries
        b = random_name_and_stat()
        b_norm = sqrt(b.h^2 + b.f^2 + b.l^2 + b.t^2)
        score = (a.h * b.h + a.f * b.f + a.l * b.l + a.t * b.t)/b_norm
        if score > best_score
            best_score = score
            best_b = b
        end
    end
    return Fighter(best_b.name, a.h, a.f, a.l, a.t)
end

function eval_battle_list(a::Fighter, bs::Array{Fighter})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function eval_battle_list2(a::Fighter, bs::Array{Fighter}, w::Vector{Float64})::AbstractFloat
    score = 0.0
    for ii in 1:length(bs)
        score = score + w[ii] * eval_battle(a, bs[ii])
    end
    return score
end


function pick_best(as::Array{Fighter}, bs::Array{Fighter})::Fighter
    bestscore = -999
    bestf = as[1]
    for ii in 1:length(as)
        score = eval_battle_list(as[ii], bs)
        if score > bestscore
            bestscore = score
            bestf = as[ii]
        end
    end
    return bestf
end

function pick_best_rdmly(as::Array{Fighter}, bs::Array{Fighter}, ntries::Int)::Fighter
    bestscore = -999
    bestf = rand(as)
    for ii in 1:ntries
        f = rand(as)
        score = eval_battle_list(f, bs)
        if score > bestscore
            bestscore = score
            bestf = f
        end
    end
    return bestf
end

function pick_best_rdmly_g(g::Function, bs::Array{Fighter}, ntries::Int)::Fighter
    bestscore = -999
    bestf = g()
    for ii in 1:ntries
        f = g()
        score = eval_battle_list(f, bs)
        if score > bestscore
            bestscore = score
            bestf = f
        end
    end
    return bestf
end

function fighters_to_df(as::Array{Fighter})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ts = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ts[ii] = as[ii].t    
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, t = ts)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

function eval_team_battle(as::Array{Fighter}, bs::Array{Fighter})::Int
    a_i = 1
    b_i = 1
    while (a_i <= length(as)) && (b_i <= length(bs))
        res = eval_battle(as[a_i], bs[b_i])
        if res == 1
            b_i = b_i + 1
        else
            a_i = a_i + 1
            if res == 0
                b_i = b_i + 1
            end
        end
    end
    a_out = (a_i > length(as))
    b_out = (b_i > length(as))
    if a_out
        if b_out
            return 0
        else
            return -1
        end
    else
        return 1
    end
end

function compare_generator(f1, f2, limit)
    a_i = 1
    b_i = 1
    f_a = f1()
    f_b = f2()
    while (a_i < limit) && (b_i < limit)
        res = eval_battle(f_a, f_b)
        if res != -1
            b_i = b_i + 1
            f_b = f2()
        end
        if res != 1
            a_i = a_i + 1
            f_a = f1()
        end
    end
    return (a_i/limit, b_i/limit)
end

function random_team(f::Function, team_size::Int)::Array{Fighter}
    team = Array{Fighter}(undef, team_size)
    for i in 1:team_size
        team[i] = f()
    end
    return team
end

random_team (generic function with 1 method)

In [2]:
20 * 20 * 20 * 20

160000

In [3]:
library = Array{Fighter}(undef, 160000)
f_i = 1
for h in MNV:MXV
    for f in MNV:MXV
        for l in MNV:MXV
            for t in MNV:MXV
                if (h * f + h*l + f*t) <= MXS && (h * f + h*l + f*t + min(h, f) > MXS)
                    ff = rand_rename(Fighter(" ", h, f, l, t), 100)
                    library[f_i] = ff
                    f_i = f_i + 1
                end
            end
        end
    end
end
library = library[1:(f_i-1)]

770-element Vector{Fighter}:
 Fighter("Trampoline Talilah", 1, 4, 16, 20)
 Fighter("Hot Zecharia", 1, 4, 20, 19)
 Fighter("Rough Trampoline", 1, 5, 5, 18)
 Fighter("Giant Joker", 1, 5, 10, 17)
 Fighter("Vonn the Trampoline", 1, 5, 15, 16)
 Fighter("Politician Jet", 1, 5, 20, 15)
 Fighter("Quilled Engine", 1, 6, 4, 15)
 Fighter("Brutal Trampoline", 1, 6, 10, 14)
 Fighter("Friendly Xerox", 1, 6, 16, 13)
 Fighter("Noor the Rough", 1, 7, 2, 13)
 Fighter("Zinc Trampoline", 1, 7, 9, 12)
 Fighter("Conscientious Cobbler", 1, 7, 16, 11)
 Fighter("Jayloni the Warrior", 1, 8, 4, 11)
 ⋮
 Fighter("Keana the Cloud", 18, 3, 2, 3)
 Fighter("Horeb the Turbo", 18, 4, 1, 2)
 Fighter("Cloud Emberlynn", 19, 1, 4, 5)
 Fighter("Zebra Anze", 19, 2, 2, 12)
 Fighter("Adya the Agile", 19, 2, 3, 2)
 Fighter("Elusive Snake", 19, 3, 1, 8)
 Fighter("Superior the Silent", 19, 3, 2, 1)
 Fighter("Luxon the Silent", 19, 4, 1, 1)
 Fighter("Unreliable Gunther", 20, 1, 3, 20)
 Fighter("Unreliable Thia", 20, 2, 1, 20)
 Figh

In [4]:
# # check ties
# for i in 1:length(library)
#     for j in 1:length(library)
#         if i!= j
#             if eval_battle(library[i], library[j]) ==0
#                 println(library[i])
#                 println(library[j])
#                 println(" ")
#             end
#         end
#     end
# end

## Nash prob dist

In [69]:
n_fs = length(library)
i_lose = Array{Int}(undef, (n_fs, n_fs))
n_lose = Array{Int}(undef, n_fs)
for i in 1:n_fs
    n_lose[i] = 0
    ff = library[i]
    for j in 1:n_fs
        if eval_battle(ff, library[j]) ==-1
            n_lose[i] += 1
            i_lose[i, n_lose[i]] = j
        end
    end
end

In [70]:
# initialize FP with 1 of each
counts = [0 for i in 1:n_fs]
wins = [0 for i in 1:n_fs]
for i in 1:n_fs
    counts[i] += 1
    for j in 1:n_lose[i]
        i_w = i_lose[i, j]
        wins[i_w]+= 1
    end
end

In [96]:
nits = 200000
for iter in 1:nits
    ind_winners = findall(wins .== maximum(wins))
    for i in ind_winners
        counts[i] += 1
        for j in 1:n_lose[i]
            i_w = i_lose[i, j]
            wins[i_w]+= 1
        end
    end
    if mod1(iter, 100000) == 1
        mxv, mxi = findmax(wins)
        print(library[mxi])
        print(" n:")
        print(counts[mxi])
        print(" w:")
        print(wins[mxi])
        println()
        mxv, mxi = findmax(counts)
        print(library[mxi])
        print(" n:")
        print(counts[mxi])
        print(" w:")
        print(wins[mxi])
        println()
        println()
    end
end

w = counts./sum(counts)
maximum([eval_battle_list2(ff, library, w) for ff in library])

Fighter("Philosopher Kiren", 7, 1, 13, 2) n:114410 w:3479029
Fighter("Naughty Kaliel", 9, 9, 1, 1) n:191893 w:3479027

Fighter("Heron Asanti", 3, 11, 7, 4) n:56532 w:3579121
Fighter("Naughty Kaliel", 9, 9, 1, 1) n:197410 w:3579116



0.017737599071819453

In [102]:
770/sum(counts)

0.00010363970531328465

In [108]:
# FP is slow to converge in the end
eps = 0.00001/length(library)
nits = 10000000
scores = wins./sum(counts)
w = counts./sum(counts)
0

0

In [109]:

for iter in 1:nits
    ind_winners = findall(wins .== maximum(wins))
    i = rand(ind_winners)
    w[i] += eps
    for j in 1:n_lose[i]
        i_w = i_lose[i, j]
        scores[i_w]+= eps
    end
    scores = scores./sum(w)
    w = w./sum(w)
    if mod1(iter, 1000000) == 1
        mxv, mxi = findmax(wins)
        print(library[mxi])
        print(" n:")
        print(w[mxi])
        print(" w:")
        print(scores[mxi])
        println()
        mxv, mxi = findmax(counts)
        print(library[mxi])
        print(" n:")
        print(w[mxi])
        print(" w:")
        print(scores[mxi])
        println()
        println(maximum([eval_battle_list2(ff, library, w) for ff in library]))
        println()
    end
end


Fighter("Xtraordinary Violin", 2, 10, 10, 6) n:0.021081136356073404 w:0.49521217567574866
Fighter("Naughty Kaliel", 9, 9, 1, 1) n:0.02731431127909478 w:0.49521163728767610.017737585854448193


Fighter("Xtraordinary Violin", 2, 10, 10, 6) n:0.025092511623185175 w:0.49744208981611104
Fighter("Naughty Kaliel", 9, 9, 1, 1) n:0.02696187347681186 w:0.488821898980570270.01997669128464844


Fighter("Xtraordinary Violin", 2, 10, 10, 6) n:0.029060830031500836 w:0.4996345291739838
Fighter("Naughty Kaliel", 9, 9, 1, 1) n:0.026613983195548512 w:0.48251460775802680.028329888436939163


Fighter("Xtraordinary Violin", 2, 10, 10, 6) n:0.03298704384021808 w:0.5017895806243243
Fighter("Naughty Kaliel", 9, 9, 1, 1) n:0.02627058175844283 w:0.47628869980157370.03979194490417728




LoadError: InterruptException: